在鐵達尼資料集中，以 Titanic_train.csv 中，首先將有遺失值的數值刪除，試著將課堂中所學的方法應用上去。

Q1：目標變數為 Survived，以 Pclass, Age, SibSp, Parch, Fare,Sex, Embarked 為特徵，試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性。

In [20]:
import numpy as np
import seaborn as sns
import pandas as pd
import sklearn

from IPython.display import display

# 特徵選取會用到的函數
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing

from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

In [3]:
data = pd.read_csv('train.csv')
data = data.dropna()
data.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age            False
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin          False
Embarked       False
dtype: bool

In [4]:
N_data = data[['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex', 'Embarked']]
N_data['Survived_cate'] = data['Survived']
N_data['Survived_cate'] = N_data['Survived_cate'].astype('object')
N_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183 entries, 1 to 889
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Survived       183 non-null    int64  
 1   Pclass         183 non-null    int64  
 2   Age            183 non-null    float64
 3   SibSp          183 non-null    int64  
 4   Parch          183 non-null    int64  
 5   Fare           183 non-null    float64
 6   Sex            183 non-null    object 
 7   Embarked       183 non-null    object 
 8   Survived_cate  183 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 14.3+ KB


<ipython-input-4-fa9f918877e5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  N_data['Survived_cate'] = data['Survived']
<ipython-input-4-fa9f918877e5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  N_data['Survived_cate'] = N_data['Survived_cate'].astype('object')


In [8]:
num_feature = []
for dtype, feature in zip(N_data.dtypes, N_data.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_feature.append(feature)

print(f'{len(num_feature)} Numberic Features : {num_feature}')

6 Numberic Features : ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']


In [13]:
cat_feature = []
for dtype, feature in zip(N_data.dtypes, N_data.columns):
    if dtype == 'object':
        cat_feature.append(feature)

print(f'{len(cat_feature)} category Features : {cat_feature}')

3 category Features : ['Sex', 'Embarked', 'Survived_cate']


In [14]:
x = N_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex', 'Embarked']]
y = N_data['Survived']
display(x.head())
display(y.head())

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
1,1,38.0,1,0,71.2833,female,C
3,1,35.0,1,0,53.1000,female,S
6,1,54.0,0,0,51.8625,male,S
10,3,4.0,1,1,16.7000,female,S
11,1,58.0,0,0,26.5500,female,S


1     1
3     1
6     0
10    1
11    1
Name: Survived, dtype: int64

In [16]:
sex_map = {'male' : 1, 'female' : 0}
N_data['Sex_one_hot'] = N_data['Sex'].map(sex_map)

embarked_map = {'C' : 0, 'Q' : 1, 'S' : 2}
N_data['Embarked_one_hot'] = N_data['Embarked'].map(embarked_map)

x_data = N_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_one_hot', 'Embarked_one_hot']]
x_data.head()

<ipython-input-16-4f504bb9acd1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  N_data['Sex_one_hot'] = N_data['Sex'].map(sex_map)


,Pclass,Age,SibSp,Parch,Fare,Sex_one_hot,Embarked_one_hot
1,1,38.0,1,0,71.2833,0,0
3,1,35.0,1,0,53.1000,0,2
6,1,54.0,0,0,51.8625,1,2
10,3,4.0,1,1,16.7000,0,2
11,1,58.0,0,0,26.5500,0,2


In [25]:
estimator = SVC(kernel = "linear")
selector = RFE(estimator, n_features_to_select=2, step=1)
selector = selector.fit(x_data, y)

print(selector.support_)
print(selector.ranking_)
print(x_data.loc[:, selector.support_].columns.tolist())

[False False  True False False  True False]
[2 4 1 5 6 1 3]
['SibSp', 'Sex_one_hot']
